In [ ]:
%load_ext autoreload
%autoreload 2
from lib import load, timing

YEAR = 2024
DAY = 7
TESTDATA = [
    None,
    '190: 10 19\n3267: 81 40 27\n83: 17 5\n156: 15 6\n7290: 6 8 6 15\n161011: 16 10 13\n192: 17 8 14\n21037: 9 7 18 13\n292: 11 6 16 20\n')'
][0]
TEST = TESTDATA is None

In [ ]:
@timing
def prepare_data():
    data = load(YEAR, DAY, split_lines=True, test=TESTDATA if TEST else None)
    sep = [row.split(':') for row in data['split']]
    return [{'result': int(a), 'numbers': [int(token) for token in b.strip().split()]} for a, b in sep]


In [ ]:
from math import log10

def concat_numbers(first, second):
    places2nd = int(log10(second)) + 1 if second > 0 else 1
    return first * (10**places2nd) + second

assert concat_numbers(0, 0) == 0
assert concat_numbers(0, 1) == 1
assert concat_numbers(0, 99) == 99
assert concat_numbers(1, 0) == 10
assert concat_numbers(99, 0) == 990
assert concat_numbers(1, 10) == 110
assert concat_numbers(88, 99) == 8899


def is_equal(numbers, operations, desired):
    result = numbers[0]
    for (op, num) in zip(operations, numbers[1:]):
        if op == 0:
            result += num
        elif op == 1:
            result *= num
        elif op == 2:
            result = concat_numbers(result, num)
        else:
            raise ParameterError(f'operation {op} unknown!')
        if result > desired:
            return False
    return result == desired

assert is_equal([5], [], 5)
assert is_equal([5, 3], [0], 8)
assert is_equal([5, 3], [1], 15)
assert is_equal([5, 3, 4], [0, 1], 32)
assert is_equal([5, 3, 4], [1, 0], 19)
assert is_equal([5, 3, 4], [2, 0], 57)
assert is_equal([5, 3, 4], [0, 2], 84)


def is_valid_row_level1(result, numbers):
    n = len(numbers) - 1
    for operation_code in range(2 ** n):
        ops = [1 & (operation_code >> i) for i in range(n)]
        if is_equal(numbers, ops, result):
            return True
    return False

assert not is_valid_row_level1(18, [5, 3, 4])
assert is_valid_row_level1(12, [5, 3, 4])
assert is_valid_row_level1(19, [5, 3, 4])
assert is_valid_row_level1(32, [5, 3, 4])
assert is_valid_row_level1(60, [5, 3, 4])


def is_valid_row_level2(result, numbers):
    n = len(numbers) - 1
    for operation_code in range(3 ** n):
        ops = [(operation_code // (3 ** i)) % 3 for i in range(n)]
        if is_equal(numbers, ops, result):
            return True
    return False

assert not is_valid_row_level2(18, [5, 3, 4])
assert is_valid_row_level2(12, [5, 3, 4])
assert is_valid_row_level2(19, [5, 3, 4])
assert is_valid_row_level2(32, [5, 3, 4])
assert is_valid_row_level2(60, [5, 3, 4])
assert is_valid_row_level2(57, [5, 3, 4])
assert is_valid_row_level2(154, [5, 3, 4])

In [ ]:
# Level 1: Sum results of lines where the first number can be expressed as additions and mulitplications of the remaining numbers
@timing
def level1(tasks):
    results = [row['result'] for row in tasks if is_valid_row_level1(**row)]
    return sum(results)


tasks = prepare_data()
print(level1(tasks))

In [ ]:
# Level 2: Same, but a third operation is possible: concatenating two numbers
@timing
def level2(tasks):
    results = [row['result'] for row in tasks if is_valid_row_level1(**row) or is_valid_row_level2(**row)]
    return sum(results)


tasks = prepare_data()
print(level2(tasks))